# `Jupyter` Inline Gradio Demos 🧪

Check here for some examples:  
🔗 https://colab.research.google.com/github/gradio-app/gradio/blob/main/demo/altair_plot/run.ipynb#scrollTo=Kh6lvbLkcfF2

In [1]:
import altair as alt
import gradio as gr
import numpy as np
import pandas as pd
from gradio import components
from vega_datasets import data as datasets
%load_ext gradio
%load_ext autoreload
%autoreload 2

In [2]:
# add parent folder to path
import sys
sys.path.append('..')
from gradio_blocks.charts import get_sample, human_readify, create_sales_chart

In [3]:
data = get_sample(verbose=False)
sales_chart = create_sales_chart(data)

In [4]:
def create_stock_chart():
    """Sample chart from vega_datasets with interactivity"""
    source = datasets.stocks()
    highlight = alt.selection_point(on='mouseover', fields=['symbol'], nearest=True)
    base = alt.Chart(source).encode(x='date:T', y='price:Q', color='symbol:N')
    points = base.mark_circle() \
                 .encode(opacity=alt.value(0)) \
                 .add_params(highlight) \
                 .properties(width=600) 
    lines = base.mark_line().encode(size=alt.condition(~highlight, alt.value(1), alt.value(3)))
    return points + lines

In [7]:

def plotter(plot_name):
    if plot_name == "Monthly Sales":
        data = get_sample(verbose=False)
        return create_sales_chart(data)
        
    elif plot_name == "Stock Prices":
        return create_stock_chart()


with gr.Blocks() as demo:
    gr.Markdown("# Plotter 🔦")

    button = gr.Radio(label="Plot type",
                      choices=['Monthly Sales', "Stock Prices"], value='Monthly Sales')
    plot = gr.Plot(label="Plot")

    button.change(plotter, inputs=button, outputs=[plot])
    demo.load(plotter, inputs=[button], outputs=[plot])

    input_text = gr.Textbox(placeholder="What month to highlight?")
    output_text = gr.Textbox()
    input_text.change(fn=lambda x: f"Welcome, {x}!",
        inputs=input_text,
        outputs=output_text)


if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [55]:
%%blocks

def goodbye_message(x, tone):
    if x == "":
        return ""
    elif tone == "kindness":
        return f"See you later, {x}. Take care!"
    else:  # tone == "curtness"
        return f"Get out of here {x}, you idiot!!"


def repeat_text(text, times):
    return text * times


gr.Markdown("# Tabnation 📁")

with gr.Blocks() as demo:
    with gr.Tab("💌 Welcome Message"):
        text_input = gr.Textbox(label='Name', placeholder="Enter your name..")
    
    with gr.Tab("💬 Details"):
        # Radio button for kindness or curtness
        tone_radio = gr.Radio(choices=["kindness", "curtness"], label="Tone", value="kindness")

    with gr.Tab("🎚️ Slider"):
            # A slider for the number of times to repeat the message
            repeat_slider = gr.Slider(minimum=1, maximum=10, step=1, label="Repeat", value=1)
            with gr.Accordion("Open for More!"):
                look_at_me = gr.Textbox(label="Look at me!")
                
                # Use a lambda function to update the textbox when the slider changes
                repeat_slider.change(fn=lambda value: str(value), 
                                    inputs=repeat_slider, 
                                    outputs=look_at_me)

with gr.Row():
    output_welcome = gr.Textbox(label="Welcome Message")
    output_goodbye = gr.Textbox(label="Goodbye Message")

    text_input.change(fn=lambda x: f"Welcome {x}!",
            inputs=text_input,
            outputs=output_welcome)

    # Update the goodbye message when the text input or radio button changes
    text_input.change(fn=goodbye_message,
                    inputs=[text_input, tone_radio],
                    outputs=output_goodbye)

    tone_radio.change(fn=goodbye_message,
                    inputs=[text_input, tone_radio],
                    outputs=output_goodbye)


In [57]:
#| hide
import nbdev; nbdev.nbdev_export()